## Kyso Intercom Template


### Create application on intercom workspace

Below we are importing our required libraries. We are using environment variables for improved security and smoother workflows. Follow [this guide](https://developers.intercom.com/building-apps/docs/get-started-developing-on-intercom). You will need to create an App on your Intercom workspace. Apps are like containers for the various functionality that you have access to on the Intercom Platform and each App can be managed and installed to your other Intercom workspaces independently. 


Once you have your App up & running you should be able to retrieve your Access Token. DO NOT simply copy and paste the token below. Instead, open up your bash_profile and save your token there. Don't forget to *source* the file before running the cell below.

In [32]:
import os
import requests
import pandas as pd
import json
KEY = os.environ['INTERCOM_ACCESS_TOKEN']

### Fetching the data

Now we need to access the Intercom API. You can find their API reference [here](https://developers.intercom.com/intercom-api-reference/reference#introduction). Unfortunately no examples are available in python. But we've done the work below. We start by fetching the users and returning a JSON response.

In [34]:
def get_users_page(per_page = 150, starting_after = None):
    url = 'https://api.intercom.io/contacts?per_page=%s' % (per_page)
    if starting_after:
        url += '&starting_after=%s' % (starting_after)
            
    resp = requests.get(url, headers={ 'Authorization':'Bearer %s' % KEY})
    return resp.json()
    
def get_users():
    first_page = get_users_page()
    pagination = first_page['pages']    
    _next = pagination['next']['starting_after']
    
    data = first_page['data']
    
    while _next:
        page = get_users_page(starting_after=_next)
        pagination = page['pages']
        if 'next' in pagination:
            _next = pagination['next']['starting_after']
        else:
            _next = False
            
        data.extend(page['data'])
    return data

user_data = get_users()

### Caching our data

Now we can convert all this JSON to pandas dataframes, which will make it much easier to manipulate and graph the data. Let's save everything to cache files so we dont need to download all the time. This is also handy to save data for APIs that have historical limits.

In [35]:
users = pd.DataFrame(user_data)
dict_col = users.pop('custom_attributes')
users = pd.concat([users, dict_col.apply(pd.Series)], axis=1)
users.to_json('./data/users.json')

### Fetching Company Data

Now we're doing the same as we did above, except this time for company-wide data.

In [5]:
def get_companies_page(url='https://api.intercom.io/companies'):
    resp = requests.get(url, headers={ 'Authorization':'Bearer %s' % KEY})
    return resp.json()
    
def get_companies():
    first_page = get_companies_page()
#     print(json.dumps(first_page, indent=2))
    pagination = first_page['pages']
    next_url = pagination['next']
    
    data = first_page['data']
    
    while next_url:
        page = get_companies_page(url=next_url)
        pagination = page['pages']
        if 'next' in pagination:
            next_url = pagination['next']
        else:
            next_url = False
            
        data.extend(page['data'])

    return data

company_data = get_companies()

And caching this data...

In [6]:
companies = pd.DataFrame(company_data)
companies.to_json('./data/companies.json')

And again for conversation and messaging data..

In [7]:
def get_conversations_page(url='https://api.intercom.io/conversations?order=desc&sort=updated_at'):
    resp = requests.get(url, headers={ 'Authorization':'Bearer %s' % KEY})
    return resp.json()
    
def get_conversations():
    first_page = get_conversations_page()
    pagination = first_page['pages']
    next_url = pagination['next']
    
    data = first_page['conversations']
    
    while next_url:
        page = get_conversations_page(url=next_url)
        pagination = page['pages']
        if 'next' in pagination:
            next_url = pagination['next']
        else:
            next_url = False
            
        data.extend(page['conversations'])

    return data

conversations_data = get_conversations()

In [8]:
conversations = pd.DataFrame(conversations_data)
conversations.to_json('./data/conversations.json')